In [66]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from xgboost import plot_importance, plot_tree
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
path = "/content/drive/MyDrive/Small_Data.csv"
df = pd.read_csv(path)

In [21]:
date = df['date']
station_idd = df['station_id']
df.drop(['date','station_id','TRIP ID','Unnamed: 0'],axis = 1,inplace = True)
df.head()

,dayofweek,dayofmonth,month,Longitude,Latitude,count
0,1,1,1,-87.626768,41.891377,9
1,1,1,1,-87.627734,41.884730,9
2,1,1,1,-87.656495,41.858166,2
3,1,1,1,-87.697153,41.922695,6
4,1,1,1,-87.621743,41.896910,8


In [47]:
df.rename(columns = {'count':'actual'},inplace=True)
m = int(df.shape[0]/5)
print(m)
train_X = df.iloc[0:3*m,:]
val_X =df.iloc[3*m:4*m,:]
test_X = df.iloc[4*m:,:]
a = (train_X.drop('actual',axis =1 ),train_X['actual'],val_X.drop('actual',axis =1 ),val_X['actual'],test_X.drop('actual',axis =1 ),test_X['actual'])
X_train = a[0]
y_train=a[1]
X_val=a[2]
y_val=a[3]
X_test=a[4]
Y_test=a[5]  


35291


In [48]:
val_X

,dayofweek,dayofmonth,month,Longitude,Latitude,actual
105873,0,12,8,-87.649613,41.877929,56
105874,0,12,8,-87.630585,41.875933,35
105875,0,12,8,-87.663576,41.935733,35
105876,0,12,8,-87.697200,41.910440,7
105877,0,12,8,-87.636282,41.918306,78
...,...,...,...,...,...,...
141159,4,18,10,-87.635640,41.896770,61
141160,4,18,10,-87.629791,41.879356,40
141161,4,18,10,-87.654406,41.954245,49
141162,4,18,10,-87.630585,41.875933,45


In [53]:
parameters = {'nthread':[4],
              'objective':['reg:linear'],
              'learning_rate': [ 0.05, 0.07,0.1],
              'max_depth': [5,6],
              'min_child_weight': [4,5],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [500]}

In [54]:
xgb1= xgb.XGBRegressor()
xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 2,
                        n_jobs =5,
                        verbose=True)
xgb_grid.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_val, y_val)],
        early_stopping_rounds=50,
       verbose=True)

Fitting 2 folds for each of 12 candidates, totalling 24 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  24 out of  24 | elapsed:  5.5min finished


[0]	validation_0-rmse:39.8697	validation_1-rmse:50.1753
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 50 rounds.
[1]	validation_0-rmse:39.242	validation_1-rmse:49.2353
[2]	validation_0-rmse:38.6689	validation_1-rmse:48.3847
[3]	validation_0-rmse:38.1484	validation_1-rmse:47.6464
[4]	validation_0-rmse:37.4633	validation_1-rmse:46.8661
[5]	validation_0-rmse:36.8199	validation_1-rmse:46.132
[6]	validation_0-rmse:36.2148	validation_1-rmse:45.4264
[7]	validation_0-rmse:35.667	validation_1-rmse:44.7803
[8]	validation_0-rmse:35.0279	validation_1-rmse:43.8324
[9]	validation_0-rmse:34.4286	validation_1-rmse:42.9803
[10]	validation_0-rmse:33.5449	validation_1-rmse:41.7192
[11]	validation_0-rmse:33.0411	validation_1-rmse:40.982
[12]	validation_0-rmse:32.6411	validation_1-rmse:40.5083
[13]	validation_0-rmse:32.198	validation_1-rmse:39.8416
[14]	validation_0-rmse:31.6468	validation_1-rmse:39.0191
[

GridSearchCV(cv=2, error_score=nan,
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=1, gamma=0,
                                    importance_type='gain', learning_rate=0.1,
                                    max_delta_step=0, max_depth=3,
                                    min_child_weight=1, missing=None,
                                    n_estimators=100, n_jobs=1, nthread=None,
                                    objective='reg:linear', random_state=0,
                                    reg_alpha=0, reg_...
                                    scale_pos_weight=1, seed=None, silent=None,
                                    subsample=1, verbosity=1),
             iid='deprecated', n_jobs=5,
             param_grid={'colsample_bytree': [0.7],
                         'learning_rate': [0.05, 0.07, 0.1],
                         'ma

In [70]:
X_test['pre'] = pre
X_test['actual'] = Y_test

In [69]:
r2_score(Y_test, pre)

-3.19109979406006

In [58]:
pre = xgb_grid.predict(X_test)


In [72]:
from sklearn.metrics import mean_squared_error
mean_squared_error(Y_test, pre)

1485.9727324263936

In [ ]:
pre['Prediction'] = reg.predict(X_test)
pjme_all = pd.concat([test, pre], sort=False)

In [ ]:
def f(station_id,):
  list1 = list(df.loc[df['STATION ID']==station_id].index)
  actual = df.index is in list1
  test = tes.index is in list1
  pjme_all = pd.concat([test, pre], sort=False)



In [ ]:
_ = pjme_all[['PJME_MW','MW_Prediction']].plot(figsize=(12, 5))
